In [4]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.5.5"

//Install XPlot package
#r "nuget:XPlot.Plotly.Interactive"
//#r "nuget:Microsoft.Data.Analysis"
    
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
//using Microsoft.Data.Analysis;

In [5]:
public class HeartDesease
    {
        [LoadColumn(0)]
        public float age { get; set; }
        [LoadColumn(1)]
        public float sex { get; set; }
        [LoadColumn(2)]
        public float cp { get; set; }
        [LoadColumn(3)]
        public float trestnps { get; set; }
        [LoadColumn(4)]
        public float chol { get; set; }
        [LoadColumn(5)]
        public float fbs { get; set; }
        [LoadColumn(6)]
        public float restecg { get; set; }
        [LoadColumn(7)]
        public float thalach { get; set; }
        [LoadColumn(8)]
        public float exang { get; set; }
        [LoadColumn(9)]
        public float oldpeak { get; set; }
        [LoadColumn(10)]
        public float slope { get; set; }
        [LoadColumn(11)]
        public float ca { get; set; }
        [LoadColumn(12)]
        public float thal { get; set; }
        [LoadColumn(13)]
        [ColumnName("Label")]
        //[Column(Ordinal: "13", name: "Label")]
        public bool target { get; set; }
        // public float Features { get; set; }
    }

In [6]:
display("Code for loading the data into IDataViews: training dataset and test dataset");

MLContext mlContext = new MLContext(seed: 0);

string dataPath = "heart-disease.csv";

IDataView dataView = mlContext.Data.LoadFromTextFile<HeartDesease>(dataPath, hasHeader: true, separatorChar: ',');

display("Schema of training DataView:");
display(dataView.Schema);

Code for loading the data into IDataViews: training dataset and test dataset

Schema of training DataView:

index,Name,Index,IsHidden,Type,Annotations
0,age,0,False,Single,
1,sex,1,False,Single,
2,cp,2,False,Single,
3,trestnps,3,False,Single,
4,chol,4,False,Single,
5,fbs,5,False,Single,
6,restecg,6,False,Single,
7,thalach,7,False,Single,
8,exang,8,False,Single,
9,oldpeak,9,False,Single,


In [33]:
float[] age=dataView.GetColumn<float>("age").ToArray();
var ageHistogram=Chart.Plot(new Graph.Histogram(){x=age, autobinx = false, nbinsx = 20});
//var layout=new Layout.Layout(){title="age Histogram"};
//ageHistogram.WithLayout(layout);
display(ageHistogram)
//display(age)

<!DOCTYPE html>

In [28]:
Chart.Plot(new Graph.Histogram{x = dataView.Columns["age"]})


(1,45): error CS1061: 'IDataView' does not contain a definition for 'Columns' and no accessible extension method 'Columns' accepting a first argument of type 'IDataView' could be found (are you missing a using directive or an assembly reference?)



Cell not executed: compilation error

In [13]:
var shuffledData = mlContext.Data.ShuffleRows(dataView,  seed: 2020);

var split = mlContext.Data.TrainTestSplit(dataView,  testFraction: 0.2);

var trainingData = split.TrainSet;
var testingData = split.TestSet;
var features = mlContext.Data.CreateEnumerable<HeartDesease>(trainingData, true);

display(features);//.Take(10));

index,age,sex,cp,trestnps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,57,0,1,130,236,0,0,174,0,0,1,1,2,False
1,57,0,1,130,236,0,0,174,0,0,1,1,2,False
2,57,0,1,130,236,0,0,174,0,0,1,1,2,False
3,57,0,1,130,236,0,0,174,0,0,1,1,2,False
4,57,0,1,130,236,0,0,174,0,0,1,1,2,False
5,57,0,1,130,236,0,0,174,0,0,1,1,2,False
6,57,0,1,130,236,0,0,174,0,0,1,1,2,False
7,57,0,1,130,236,0,0,174,0,0,1,1,2,False
8,57,0,1,130,236,0,0,174,0,0,1,1,2,False
9,57,0,1,130,236,0,0,174,0,0,1,1,2,False


In [16]:
display("Apply Data Transformations pipeline");

//Common data process configuration with pipeline data transformations
string[] featureColumnNames =
                dataView.Schema
                    .Select(column => column.Name)
                    .Where(columnName => columnName != "Label").ToArray();


IEstimator<ITransformer> dataPrepareEstimator = mlContext.Transforms.Concatenate("Features", featureColumnNames)
                                                            .Append(mlContext.Transforms.NormalizeMinMax("Features"))                                                                                                                        
                                                            .Append(mlContext.BinaryClassification.Trainers.AveragedPerceptron());
    
     //Train the model
            
            ITransformer model = dataPrepareEstimator.Fit(trainingData);
           

            // Use trained model to make inferences on test data
            IDataView testDataPredictions = model.Transform(testingData);
            var calibratorEstimator = mlContext.BinaryClassification.Calibrators.Platt();
            ITransformer calibratorTransformer = calibratorEstimator.Fit(testDataPredictions);
            var finalData = calibratorTransformer.Transform(testDataPredictions).Preview();
    
    CalibratedBinaryClassificationMetrics trainedModelMetrics = mlContext.BinaryClassification.Evaluate(calibratorTransformer.Transform(testDataPredictions));
     display("Accuracy" + trainedModelMetrics.Accuracy);

Apply Data Transformations pipeline

Accuracy0.8108108108108109